# Assignment 3

In [160]:
# imports
import numpy as np
import scipy.linalg as linag
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.stats import norm
from numba import njit

In [161]:
def black_scholes(S, K, T, r, sigma, type='call'):
    """Price Black-Scholes option"""

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price

In [162]:
@njit
def create_tri_diagonal_matrix(a,b,c, size):
    matrix = np.zeros(size)



    np.fill_diagonal(matrix[1:], a)  # superdiagonal
    np.fill_diagonal(matrix, b) # diagonal
    np.fill_diagonal(matrix[:, 1:], c)  # subdiagonal
    
    
    return matrix

In [163]:
test_matrix = create_tri_diagonal_matrix(1,2,3, (10,10))
print(test_matrix)

[[2. 3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 2. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 2. 3. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 2. 3. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 2. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 2. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 2. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 2.]]


Scenarios to test

In [176]:

# Scenarios to solve in assignment

# # Scenario 1
# r = 0.04
# sigma = 0.30
# S0 = 100
# K = 110
# T = 1


# # Scenario 2
# r = 0.04
# sigma = 0.30
# S0 = 110
# K = 110
# T = 1


# # Scenario 3
# r = 0.04
# sigma = 0.30
# S0 = 120
# K = 110
# T = 1

In [166]:
@njit
def FTCS_vectorized(init, M, N, r, sigma, xmin, xmax, T, q, boundary_condition = 'Dirichlet'):
    y_current = init.copy()
    y_next = np.zeros_like(y_current)
    dx = (xmax - xmin) / M
    dt = T / N

    lambda_const = 0.5 * sigma**2 * dt / dx**2

    # Checking the CFL condition value
    cfl_value = lambda_const

    if lambda_const > 0.5:
        ValueError('Warning, CFL condition indicates unstability, choose a different ratio of delta t and delta x')

    a = lambda_const
    b = 1- 2* lambda_const
    c = lambda_const


    A = create_tri_diagonal_matrix(a,b,c, (M + 1, M + 1))  # create tridiagonal matrix


    for step in range(N-1):
        if boundary_condition == 'Dirichlet':
            # Only entry we need to modify for boundary condition is j = M, as Y0 will be set by 0 by the boundary conditions and thus can be omitted.
            # Create the vector k according to the slide 25, lecture: https://canvas.uva.nl/courses/42579/files/10468265?module_item_id=1984428
            k = np.zeros(y_current.shape[0])
            tau_n = (N-1) * dt  
            # Apply the computed k value to the last element in y_next, adjusted for the boundary condition
            k[-1] = (np.exp(xmax) - np.exp(-q * tau_n)) * np.exp(0.5 * (q - 1) * xmax + 0.25 * (q + 1)**2 * tau_n)

            
            y_next = A @ y_current + k

        if boundary_condition == 'Neumann':
            # Might implement later
            pass
        
        # Prepare for the next step
        y_current = y_next.copy()
        
    return y_next


In [167]:
@njit
def Crank_Nicolson_Vectorized(init, M, N, r, sigma, xmin, xmax, T, q, boundary_condition = 'Dirichlet'):
    y_current = init.copy()
    y_next = np.zeros_like(y_current)
    dx = (xmax - xmin) / M
    dt = T / N

    lambda_const = 0.5 * sigma**2 * dt / dx**2 

    # Checking the CFL condition value
    cfl_value = lambda_const
    
    if lambda_const > 0.5:
        ValueError('Warning, CFL condition indicates unstability, choose a different ratio of delta t and delta x')

    a = -0.5 * lambda_const
    b = 1 + lambda_const
    c = -0.5 * lambda_const

    d = 0.5 * lambda_const
    e = 1 - lambda_const
    f = 0.5 * lambda_const


    A = create_tri_diagonal_matrix(a,b,c, (M + 1, M + 1))  # create tridiagonal matrix A (forward in time)

    B = create_tri_diagonal_matrix(d,e,f, (M + 1, M + 1))  # create tridiagonal matrix B (backward in time)

    for step in range(N-1):
        if boundary_condition == 'Dirichlet':
            # Only entry we need to modify for boundary condition is j = M, as Y0 will be set by 0 by the boundary conditions and thus can be omitted.
            # Create the vector k according to the slide 25, lecture: https://canvas.uva.nl/courses/42579/files/10468265?module_item_id=1984428
            k = np.zeros(y_current.shape[0])
            tau_n = (N-1) * dt  
            # Apply the computed k value to the last element in y_next, adjusted for the boundary condition
            k[-1] = (np.exp(xmax) - np.exp(-q * tau_n)) * np.exp(0.5 * (q - 1) * xmax + 0.25 * (q + 1)**2 * tau_n)

            
            # Solve the linear system A y_next = B y_current + k
            y_next = np.linalg.solve(A, B @ y_current + k)

        if boundary_condition == 'Neumann':
            # Might implement later
            pass
        
        # Prepare for the next step
        y_current = y_next.copy()
        
    return y_next


In [168]:
def transform_solution(results, r, sigma, S0, K, T, Smin, Smax, xmin, xmax, M, N, a, b, q):
    # Calculate the value of interest for S and sigma^2*T/2
    log_SK = np.log(S0 / K)
    sigma2_T2 = (sigma**2 * T) / 2

    # Generate the grid for S (in log space) and t
    x = np.linspace(xmin, xmax, M + 1)  # x grid for log(S/K)
    t_grid = np.linspace(0, T, N)   # Time grid from 0 to T


    # Interpolate in the space dimension (log(S/K))
    y_interp_func = interp1d(x, results, kind='linear') 
    y_interpolated = y_interp_func(log_SK)
    
    # No need to interpolate in time since we're evaluating at T

    # Apply the transformation formula
    V_0 = K * y_interpolated * np.exp(a * log_SK + b * sigma2_T2)

    return V_0

In [169]:
# Testing

# Scenario 1
r = 0.04
sigma = 0.30
S0 = 100
K = 110
T = 1
M = 1000
N = 1000


# Initial condition (from slide 18, lecture: https://canvas.uva.nl/courses/42579/files/10468265?module_item_id=1984428
x = np.linspace(xmin, xmax, M+1)
init = np.maximum(np.exp((q - 1) * x) - np.exp((0.5 * q - 1) * x), 0)


# Setting up the grid
Smax = 4 * K  # arbitrary choice
Smin = 0.0001  # assuming stock price can't go below 0.0001, set at small number to avoid log(0) errors
xmax = np.log(Smax / K)
xmin = np.log(Smin / K)
dx = (xmax - xmin) / M
dt = T / N
q = (2*r)/(sigma**2)
a = 0.5*(q-1)
b = 0.25*(q+1)**2

# Results FTCS
scenario_1_FCTS_results = FTCS_vectorized(init, M, N, r, sigma, xmin, xmax, T, q, boundary_condition = 'Dirichlet')

# Results Crank-Nicolson
scenario_1_Crank_results = Crank_Nicolson_Vectorized(init, M, N, r, sigma, xmin, xmax, T, q, boundary_condition = 'Dirichlet')


# Tranforming solution



In [172]:
transform_solution(scenario_1_FCTS_results,  r, sigma, S0, K, T, Smin, Smax, xmin, xmax, M, N, a, b, q)

1001
1001


3.5790737845499816

In [173]:
transform_solution(scenario_1_Crank_results, r, sigma, S0, K, T, Smin, Smax, xmin, xmax, M, N, a, b, q)

1001
1001


3.5786022972028984

In [174]:
r = 0.04
sigma = 0.30
S0 = 100
K = 110
T = 1

In [175]:
black_scholes(S0,K,T,r,sigma,type='call')

9.625357828843697

Results look quite different, don't know why